<P align="center"><img src="https://www.fs.usda.gov/Internet/FSE_MEDIA/fseprd897983.png" width = 800 height =200>

<html>
<body>
<h1 style="background-color: #022851;"><center>
    <br><font size="+3.5">
    <font color=#F8F8FF><b>Generate Catchments and PRMS Modeled Flows by Locations of Interest (LOIs) </b></font>
   </font>
    <br><font size="+1">
    <font color=#F8F8FF></b> State Water Resources Control Board </font>
   </font> 
    <br><font size="+1">
    <font color=#F8F8FF></b> Cannabis Instream Flows Unit </font> <br>
       </font> 
    <br><font size="+1">
    <font color=#F8F8FF><b>Authors:</b> Jaspreet S. Gill, P.E. , Ryan Hankins</font>
   </font> 
    </center>
</h1>
</body>
</html>

The notebook takes a user specified Digital Elevation Model (DEM) and pour points based on any Locations of Interest (LOI) to generate respective drainage catchment polygons. The notebook also takes the user specified Precipitation Runoff Modeling System (PRMS) model grid shapefile, HRU streamflow out csv, and the generated catchment areas of the specified pour points to output PRMS unimpaired flow estimates by each LOI.  All inputs can be generated with the retrieve_files.ipynb script in this repository. See the Inputs section of this notebook for more details.  Please contact Ryan Hankins at ryan.hankins@waterboards.ca.gov for any questions. 

<html>
<h2 style="background-color: #022851;">
<font size="+2"><br>
    <font color=#F8F8FF><b>&nbsp;   &nbsp; Imports and extensions</b></font>
    </font>  <br>
</h2>
</html>

In [ ]:
#import python libraries
import geopandas as gpd
import rasterio
import pandas as pd
import snowflake.connector
import sys, os
import glob


<html>
<h2 style="background-color: #022851;">
<font size="+2"><br>
    <font color=#F8F8FF><b>&nbsp;   &nbsp; Inputs</b></font>
    </font>  <br>
</h2>
</html>

In [ ]:
# CHANGE THE NAME in the quotes to match the name of the geodatabase that has the pour points shapefile, dem raster and model grid
#input_workspace = './LOI_FILES.gpkg'
input_workspace = os.getcwd()

# Change the NAME to match the large unimpaired flows by HRU file
hru_streamflow_csv = "SELECT * FROM EEL_HRU_STREAMFLOW_OUT_2002_2022" 

In [ ]:
# Turn input files into geopandas dataframe/ rasterio object

pour_points = gpd.read_file(r"example.shp") #change this to your pour points shapefile
model_grid = gpd.read_file(r"example.zip") #change this to your model grid shapefile
dem_raster = r"example.tif" # works with a .tif or .tiff file. Make sure the dem is not too large.

In [ ]:
# Set the name of the attribute column in the pour points shapefile that holds the pour point IDs

pour_point_id = "StationCod" # Change this to your pour point ID column name

<html>
<h2 style="background-color: #022851;">
<font size="+2"><br>
    <font color=#F8F8FF><b>&nbsp;   &nbsp; Projected coordinate system - NAD 1983 UTM Zone 10N</b></font>
    </font>  <br>
</h2>
</html>

In [ ]:
# Change the projection to NAD83 10N
pour_points_utmz10N = pour_points.to_crs(crs=26910)
model_grid_utmz10N = model_grid.to_crs(crs=26910)

<html>
<h2 style="background-color: #022851;">
<font size="+2"><br>
    <font color=#F8F8FF><b>&nbsp;   &nbsp; Clip pour points to watershed model grid</b></font>
    </font>  <br>
</h2>
</html>

In [ ]:

# Clip the pour_points the model grid
pour_points_clipped = gpd.clip(pour_points_utmz10N, model_grid_utmz10N)


In [ ]:
pour_points_clipped.head()

<html>
<h2 style="background-color: #022851;">
<font size="+2"><br>
    <font color=#F8F8FF><b>&nbsp;   &nbsp; Generate Pour Point Catchment Areas</b></font>
    </font>  <br>
</h2>
</html>

In [ ]:
# Read elevation raster

from pysheds.grid import Grid

grid = Grid.from_raster(dem_raster, crs = 26910) #make sure crs is in NAD83 zone 10
dem = grid.read_raster(dem_raster, crs =26910)

Condition the DEM (this may take a few minutes)

In [ ]:

# Fill pits in DEM
pit_filled_dem = grid.fill_pits(dem)

# Fill depressions in DEM
flooded_dem = grid.fill_depressions(pit_filled_dem)
    
# Resolve flats in DEM
inflated_dem = grid.resolve_flats(flooded_dem)

Calculate Flow Direction

In [ ]:
# use arcgis standard direction mapping

dirmap = (64, 128, 1, 2, 4, 8, 16, 32)
    
# Compute flow directions

fdir = grid.flowdir(inflated_dem, dirmap=dirmap)

In [ ]:
#OPTIONAL - plot flow direction map

import numpy as np
import matplotlib.pyplot as plt
from matplotlib import colors
import seaborn as sns

fig = plt.figure(figsize=(8,6))
fig.patch.set_alpha(0)

plt.imshow(fdir, extent=grid.extent, cmap='viridis', zorder=2)
boundaries = ([0] + sorted(list(dirmap)))
plt.colorbar(boundaries= boundaries,
             values=sorted(dirmap))
plt.xlabel('Longitude')
plt.ylabel('Latitude')
plt.title('Flow direction grid', size=14)
plt.grid(zorder=-1)
plt.tight_layout()


Calculate flow accumulation

In [ ]:
# Calculate flow accumulation

acc = grid.accumulation(fdir, dirmap=dirmap)

In [ ]:
# OPTIONAL - plot flow accumulation map

fig, ax = plt.subplots(figsize=(8,6))
fig.patch.set_alpha(0)
plt.grid('on', zorder=0)
im = ax.imshow(acc, extent=grid.extent, zorder=2,
               cmap='cubehelix',
               norm=colors.LogNorm(1, acc.max()),
               interpolation='bilinear')
plt.colorbar(im, ax=ax, label='Upstream Cells')
plt.title('Flow Accumulation', size=14)
plt.xlabel('Longitude')
plt.ylabel('Latitude')
plt.tight_layout()

Delineate basins and save the output to the "output_basin_rasters" folder

In [ ]:
import os
import numpy as np
import geopandas as gpd
from pysheds.grid import Grid
import rasterio
from rasterio.transform import from_origin

# Define the function to save the raster

def save_raster(array, transform, output_path, crs):
    with rasterio.open(
        output_path, 'w', driver='GTiff',
        height=array.shape[0], width=array.shape[1],
        count=1, dtype=array.dtype, crs=crs, transform=transform
    ) as dst:
        dst.write(array, 1)

# Define the function to generate the basin raster

def generate_basin_raster(pour_point, grid, fdir, output_raster_path):
    x, y = pour_point.geometry.x.values[0], pour_point.geometry.y.values[0]
    grid = Grid.from_raster(dem_raster, crs = 26910)
    # Snap pour point to high accumulation cell
    x_snap, y_snap = grid.snap_to_mask(acc > threshold, (x, y))

    if x_snap is None or y_snap is None:
        print(f"No suitable cell found for pour point: ({x}, {y})")
        return

    # Delineate the catchment
    catch = grid.catchment(x=x_snap, y=y_snap, fdir=fdir, xytype='coordinate')

    # Clip the grid to the catchment
    grid.clip_to(catch)
    catch_array = grid.view(catch)
    transform = grid.affine
    if catch_array.size == 0:
        print(f"No catchment area found for pour point: ({x_snap}, {y_snap})")
        return
    
    catch_array_numeric = catch_array.astype(np.uint8)

    # Save raster
    crs = "EPSG:26910"  # Adjust CRS as needed
    save_raster(catch_array_numeric, transform, output_raster_path, crs)



In [ ]:

# Directory setup
output_dir = 'output_basin_rasters'
os.makedirs(output_dir, exist_ok=True)

# Ensure the GeoDataFrame is in the correct CRS 
pour_points_clipped = pour_points_clipped.to_crs(epsg=26910)  


# Iterate through each unique 'stationcod'
for SITENO in pour_points_clipped[f'{pour_point_id}'].unique():
    # Filter the pour points for the current SITENO
    pour_point = pour_points_clipped[pour_points_clipped[f'{pour_point_id}'] == SITENO]
    if pour_point.empty:
        print(f"No pour points found for SITENO: {SITENO}")
        continue
    
    # Generate the output raster file path
    output_raster_path = os.path.join(output_dir, f"{SITENO}.tif")
    # Call the function to generate the basin raster
    generate_basin_raster(pour_point, grid, fdir, output_raster_path)
    
    print(f"Raster for SITENO {SITENO} saved to {output_raster_path}")

print("All rasters generated.")

<html>
<h2 style="background-color: #022851;">
<font size="+2"><br>
    <font color=#F8F8FF><b>&nbsp;   &nbsp; Spatial Join HRUs with Pour Point Catchment Areas</b></font>
    </font>  <br>
</h2>
</html>

In [ ]:
import os
import rasterio
import geopandas as gpd
from shapely.geometry import shape
from rasterio.features import shapes

# Path to the folder containing the raster files
folder_path = 'output_basin_rasters'

# List to store the geometries and file names
geometries = []
file_names = []

# Loop through the folder and process each raster file
for file_name in os.listdir(folder_path):
    if file_name.endswith('.tif'):  # Adjust extension as necessary
        file_path = os.path.join(folder_path, file_name)
        
        with rasterio.open(file_path) as src:
            # Read the raster data
            image = src.read(1)  # Read the first band
            
            # Use rasterio's shapes function to create polygons from the raster
            # We only want shapes where the value is 1
            mask = image == 1
            shapes_gen = shapes(image, mask=mask, transform=src.transform)
            
            # Create a list to store individual shapes
            for geom, value in shapes_gen:
                if value == 1:  # Ensure the shape is for value 1
                    geom = shape(geom)  # Convert shape to a Shapely geometry
                    geometries.append(geom)
                    file_names.append(os.path.splitext(file_name)[0])
        
# Create a GeoDataFrame from the lists
gdf = gpd.GeoDataFrame({
    'filename': file_names,
    'geometry': geometries
}, crs=src.crs)  # Use the CRS of the raster file

gdf = gdf.rename(columns={"filename": "SITENO"})

# Print the GeoDataFrame to check the result
print(gdf)


In [ ]:
#save output to geopackage

gdf.to_file("generated_basins.gpkg", layer = "generated_basins", driver='GPKG')
pour_points_clipped.to_file("generated_basins.gpkg", layer = "pour_points", driver='GPKG')

In [ ]:
# Plot Generated Basins and Pour Points
import matplotlib.pyplot as plt
fig, ax = plt.subplots(figsize=(10, 10))

# Plot the first GeoDataFrame
gdf.plot(ax=ax, color='blue', alpha=0.5, edgecolor='k', label='Layer 1')

# Plot the second GeoDataFrame
pour_points_clipped.plot(ax=ax, color='red', alpha=0.5, edgecolor='k', label='Layer 2')

ax.legend()

# Set title and labels if needed
ax.set_title('LOIs and Generated Basins')
ax.set_xlabel('Longitude')
ax.set_ylabel('Latitude')

# Show the plot
plt.show()

In [ ]:
# Spatial Join model grid and generated basins for flow calculations
master_hrus_by_LOIs = gpd.sjoin(model_grid_utmz10N,gdf, how='inner', predicate='intersects')
master_hrus_by_LOIs.head()

<html>
<h2 style="background-color: #022851;">
<font size="+2"><br>
    <font color=#F8F8FF><b>&nbsp;   &nbsp; Create CSVs of each LOI with HRU IDs, Subbasin and StationCod</b></font>
    </font>  <br>
</h2>
</html>

In [ ]:
# Specify the folder name
folder_name = "LOI_csvs"

# Get the current working directory
current_directory = os.getcwd()

# Create the full path to the new folder
input_csvs = os.path.join(current_directory, folder_name)


# Check if the folder exists
if not os.path.exists(input_csvs):
    # If not, create the folder
    os.makedirs(input_csvs)
    print(f"Folder '{input_csvs}' created.")
else:
    print(f"Folder '{input_csvs}' already exists.")

In [ ]:
# Group by the pour point ID in the beginning and save each group as a separate CSV file
master_hrus_by_LOIs = master_hrus_by_LOIs.groupby("SITENO")


In [ ]:
# save the LOI and HRU IDs csv files
master_hrus_by_LOIs.apply(lambda group: pd.DataFrame.to_csv(group, os.path.join(input_csvs, f'{group.name}.csv')))

In [ ]:
# This block of code reads the csvs back, filters necessary columns and saves back to the same folder
# Specify the file extension to filter for CSV files
ext = ('.csv')

# Loop through each file in the folder and process them
for filename in os.listdir(input_csvs):
    f = os.path.join(input_csvs, filename)
    
    # Check if the file has the specified extension
    if f.endswith(ext):
        # Split the file path to get the filename without extension
        head_tail = os.path.split(f)
        head_tail1 = input_csvs
        k = head_tail[1]
        r = k.split(".")[0]
          
        # Create a new file path for the processed CSV
        p = os.path.join(head_tail1, f"{r}.csv")

        # Read the CSV file
        mydata = pd.read_csv(f)
            
        # Extract specific columns (pour_point_id, 'SUB_BASIN', 'HRU_ID') and save to a new CSV
        new = mydata[["SITENO", 'sub_basin', 'hru_id']]
        new.to_csv(p, index=False)


<html>
<h2 style="background-color: #022851;">
<font size="+2"><br>
    <font color=#F8F8FF><b>&nbsp;   &nbsp; Read Only Active HRU IDs from Model Grid Feature Class</b></font>
    </font>  <br>
</h2>
</html>

In [ ]:

# Specify the field (column) name
field_name = "hru_id"
filter_columns = model_grid[field_name].tolist()

# Now, field_values is a list containing all the values from the specified column
print(filter_columns[1:10])

<html>
<h2 style="background-color: #022851;">
<font size="+2"><br>
    <font color=#F8F8FF><b>&nbsp;   &nbsp; Read HRU Streamflow Out File</b></font>
    </font>  <br>
</h2>
</html>

In [ ]:
# If you do not have access to snowflake, read the stream flow csv manually
# hrus = pd.read_csv("EEL_HRU_STREAMFLOW_OUT_2002_2022.csv") #change this to your streamflow csv
# Fill in your snowflake connection parameters and retrieve the master hru streamflow table
conn = snowflake.connector.connect( 
    account="", # account name is found in the Single Sign On URL
    user="", #your waterboards email
    authenticator="externalbrowser",
    role="DWR_DEV_RWC_SFROLE", 
    warehouse="COMPUTE_WH",  
    database="DWR_DEV",  
    schema="INSTREAM_FLOWS_NC_DEV")

cur = conn.cursor()

In [ ]:
cur.execute(hru_streamflow_csv)

In [ ]:
# Turn streamflow table into a pandas dataframe for further manipulation.
hrus = cur.fetch_pandas_all()

In [ ]:
# Check to make sure the last date is present and columns got read properly
hrus_filtered = hrus.rename(columns={'HRU_ID': 'hru_id'})
hrus_filtered.columns = hrus_filtered.columns.str.strip()
hrus_filtered["hru_id"] = hrus_filtered["hru_id"].astype(int)
hrus_filtered.set_index("hru_id", inplace=True)

<html>
<h2 style="background-color: #022851;">
<font size="+2"><br>
    <font color=#F8F8FF><b>&nbsp;   &nbsp; Output Modeled PRMS Unimpaired Flows by LOIs</b></font>
    </font>  <br>
</h2>
</html>

In [ ]:
# Check the input_csvs folder to see if it exists
input_csvs = "LOI_csvs"

In [ ]:
# Read the input csvs to a list so we can create a flows table for each one
import glob
path = os.getcwd()
csv_files = glob.glob(os.path.join(path, input_csvs, "*.csv"))
print(csv_files)

In [ ]:
# Specify the folder name
output_folder = "outFlows_by_LOIs"

# Get the current working directory
current_directory = os.getcwd()

# Create the full path to the new folder
output_flows_folder = os.path.join(current_directory, output_folder)

# Check if the folder exists
if not os.path.exists(output_flows_folder):
    # If not, create the folder
    os.makedirs(output_flows_folder)
    print(f"Folder '{output_flows_folder}' created.")
else:
    print(f"Folder '{output_flows_folder}' already exists.")

In [ ]:
import os
import pandas as pd

# Assuming csv_files is a list of file paths and hrus_filtered is a DataFrame

# Create a list of file names from the LOI_csvs folder
fns = [os.path.splitext(os.path.basename(x))[0] for x in csv_files]

# Read the csv files into dataframes
d = {fn: pd.read_csv(csv_file) for fn, csv_file in zip(fns, csv_files)}

# Extract HRU IDs from dataframes
l = {key: value['hru_id'].astype(int).tolist() for key, value in d.items()}

# Initialize an empty dictionary to store the results
s = {}

# Process HRU data to sum all rows in each column
for key, value in l.items():
    valid_ids = [vid for vid in value if vid in hrus_filtered.index]
    if valid_ids:  # Only process if there are valid IDs
        df_subset = hrus_filtered.loc[valid_ids]
        # Compute the sum of all rows in each column
        column_sums = df_subset.sum(axis=0)
        # Store the result in the dictionary s
        s[key] = pd.DataFrame(column_sums).T  # Transpose to get the sums in a single row

# Combine all DataFrames into a single DataFrame
combined_df = pd.concat(s.values(), ignore_index=True)
combined_df.index = s.keys()  # Set index to be the keys from the original dictionary

# Save the combined DataFrame to a single CSV file
output_file = os.path.join(output_folder, "combined_flows_cfs.csv")
combined_df.to_csv(output_file)
print(f"Saved combined data to {output_file}")
